# check jetson nano

In [1]:
import subprocess
import pkg_resources
import platform
import os, sys
import torch, tensorflow

In [2]:
# print model
with open('/proc/device-tree/model', 'r') as f:
    print(str(f.read()[:-1]))

NVIDIA Jetson Nano Developer Kit


In [3]:
# network connection(s)
for interface in ['eth0', 'wlan0']:
    operstate = subprocess.check_output('cat /sys/class/net/%s/operstate' % interface, shell=True).decode('ascii')[:-1]
    if operstate == 'up' :  
        cmd = "ifconfig %s | grep -Eo 'inet (addr:)?([0-9]*\.){3}[0-9]*' | grep -Eo '([0-9]*\.){3}[0-9]*' | grep -v '127.0.0.1'" % interface
        ip = subprocess.check_output(cmd, shell=True).decode('ascii')[:-1]
    else:
        ip = ''
    print(f'{interface}\t{operstate}\t{ip}')

eth0	up	192.168.178.90
wlan0	up	192.168.178.71


In [4]:
print('python:     ', sys.version.replace('\n', ' '))
print('pytorch:    ', torch.__version__)
print('keras :     ', tensorflow.keras.__version__)
print('tensorflow: ', tensorflow.__version__)
# print('CUDA', "tbd ")
print('CUDNN:      ', torch.backends.cudnn.version())
#print('number of CUDA sevices:', torch.cuda.device_count())
#print('num GPUs available:' , len(tensorflow.config.experimental.list_physical_devices('GPU')))
#print('devices')
# !nvidia-smi
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
#print('active CUDA device gpu', torch.cuda.current_device())
#print ('available devices ', torch.cuda.device_count())
#print ('current cuda device ', torch.cuda.current_device())

python:      3.6.9 (default, Oct  8 2020, 12:12:24)  [GCC 8.4.0]
pytorch:     1.3.0
keras :      2.4.0
tensorflow:  2.3.1
CUDNN:       7603


In [7]:
# input devices (e.g. to detect devicename of a gamepad)
!ls /dev/input/event*
print('###')
!cat /proc/bus/input/devices

/dev/input/event0  /dev/input/event1  /dev/input/event2  /dev/input/event3
###
I: Bus=0000 Vendor=0000 Product=0000 Version=0000
N: Name="tegra-hda HDMI/DP,pcm=3"
P: Phys=ALSA
S: Sysfs=/devices/70030000.hda/sound/card0/input0
U: Uniq=
E: Enabled=0
H: Handlers=event0 
B: PROP=0
B: EV=21
B: SW=140

I: Bus=0019 Vendor=0001 Product=0001 Version=0100
N: Name="gpio-keys"
P: Phys=gpio-keys/input0
S: Sysfs=/devices/gpio-keys/input/input1
U: Uniq=
E: Enabled=0
H: Handlers=kbd event1 
B: PROP=0
B: EV=3
B: KEY=10000000000000 0

I: Bus=0003 Vendor=413c Product=2106 Version=0110
N: Name="Dell Dell QuietKey Keyboard"
P: Phys=usb-70090000.xusb-2.1/input0
S: Sysfs=/devices/70090000.xusb/usb1/1-2/1-2.1/1-2.1:1.0/0003:413C:2106.0001/input/input2
U: Uniq=
E: Enabled=0
H: Handlers=sysrq kbd leds event2 
B: PROP=0
B: EV=120013
B: KEY=1000000000007 ff9f207ac14057ff febeffdfffefffff fffffffffffffffe
B: MSC=10
B: LED=7

I: Bus=0003 Vendor=046d Product=c05a Version=0111
N: Name="Logitech USB Optical Mouse"
P: 

In [6]:
# cameras
!ls /dev/ 

autofs			mmcblk0p9	    ram7	       tty51
block			mqueue		    ram8	       tty52
btrfs-control		mtd0		    ram9	       tty53
bus			mtd0ro		    random	       tty54
char			mtdblock0	    rfkill	       tty55
console			net		    rtc		       tty56
constraint_cpu_freq	network_latency     rtc0	       tty57
constraint_gpu_freq	network_throughput  rtc1	       tty58
constraint_online_cpus	null		    shm		       tty59
cpu_dma_latency		nvhdcp0		    snd		       tty6
cpu_freq_max		nvhdcp1		    spidev0.0	       tty60
cpu_freq_min		nvhost-as-gpu	    spidev1.0	       tty61
cuse			nvhost-ctrl	    stderr	       tty62
disk			nvhost-ctrl-gpu     stdin	       tty63
emc_freq_min		nvhost-ctrl-isp     stdout	       tty7
fb0			nvhost-ctrl-isp.1   tegra_camera_ctrl  tty8
fb1			nvhost-ctrl-nvdec   tegra_cec	       tty9
fd			nvhost-ctrl-vi	    tegra-crypto       ttyGS0
full			nvhost-ctxsw-gpu    tegra_dc_0	       ttyp0
fuse			nvhost-dbg-gpu	    tegra_dc_1	       ttyp1
gpiochip0		nvhost-gpu	    tegra_dc_ctrl      ttyp2
g

# jetbot only

In [7]:
# check i2c bus
#  found 0x3c : display Adafruit_SSD1306 
#  found 0x48 : ads1115 -> analog digital converter to get voltages e.g. the 12V on jetbot platine
#  found 0x60 : motor driver : Adafruit_MotorHAT
#  found 0x70 : ?
for i in range(128):
    address = str(hex(i))
    found_address = os.popen("i2cdetect -y -r 1 " + address + " " + address + " | egrep '" + address[2:] + "' | awk '{print $2}'").read()
    if address[2:] in found_address:
        print('found', address)

found 0x3c
found 0x48
found 0x60
found 0x70


In [8]:
# ina219 should return values charging. in my case on this i2c address is nothing found
import jetbot.ina219 as ina219 # import INA219
adress = os.popen("i2cdetect -y -r 1 0x41 0x41 | egrep '41' | awk '{print $2}'").read()
if(adress=='41\n'):
    ina = ina219.INA219(addr=0x41)
else:
    ina = None
    
print(ina)

None


In [9]:
# check power supply on jetbot
# ch 0,1   :  ~9,7 V
# ch 2     :  ~12 V (only if power supply is connected)
# ch 3     :  0 V
# ch 4     :  ~12 V (also if battery only without power supply)
# ch 5,6,7 :  ~2,4V (i2c?)

import jetbot.ads1115 as ads1115 # import ADS1115
adress = os.popen("i2cdetect -y -r 1 0x48 0x48 | egrep '48' | awk '{print $2}'").read()
if(adress=='48\n'):
    ads = ads1115.ADS1115()
    for channel in range(8):
        print(f'channel {channel} {ads.readVoltage(channel)/1000:.2f} volt')
else:
    ads = None

channel 0 9.50 volt
channel 1 9.50 volt
channel 2 0.00 volt
channel 3 0.00 volt
channel 4 12.08 volt
channel 5 2.37 volt
channel 6 2.37 volt
channel 7 2.37 volt
